# 05 GSEA+Cytoscape

## Notebook setup

In [1]:
import scanpy as sc
import scanpy.external as sce
import numpy as np
import pandas as pd
import warnings, scipy.sparse as sp, matplotlib, matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.pyplot import rc_context
from collections import Counter
import matplotlib.font_manager
import openpyxl
import pyreadr
import rpy2
from rpy2.robjects.packages import importr
import rpy2.robjects as robjects
#import magic
#import seaborn as sns
import palantir
import loompy
import feather
import re
#from scipy.sparse import csgraph

matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
matplotlib.rcParams['font.family'] = 'sans-serif'
matplotlib.rcParams['font.sans-serif'] = 'Arial'
matplotlib.rc('font', size=14)
import matplotlib.lines as lines

pd.set_option('display.max_rows', 200)

sc.set_figure_params(dpi=80, dpi_save=300, color_map='Spectral_r', vector_friendly=True, transparent=True)
sc.settings.verbosity = 3 # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()

warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

findfont: Font family ['Raleway'] not found. Falling back to DejaVu Sans.
findfont: Font family ['Lato'] not found. Falling back to DejaVu Sans.
/Users/akousa/opt/anaconda3/envs/KousaJahnZhao_et_al_2023/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


scanpy==1.9.1 anndata==0.8.0 umap==0.5.3 numpy==1.20.2 scipy==1.5.3 pandas==1.4.4 scikit-learn==1.1.2 statsmodels==0.13.2 python-igraph==0.9.11 pynndescent==0.5.7


In [2]:
user_defined_palette =  [ '#F6222E', '#FEAF16','#3283FE','#BDCDFF', '#3B00FB', '#F8A19F', '#1CFFCE',  '#C4451C', 
                          '#2ED9FF', '#c1c119', '#8b0000', '#FE00FA', '#1CBE4F','#B5EFB5', '#0e452b', '#AA0DFE']

In [3]:
user_defined_cmap_markers = LinearSegmentedColormap.from_list('mycmap', ["#E6E6FF", "#CCCCFF", "#B2B2FF", "#9999FF",  "#6666FF",   "#3333FF", "#0000FF"])
user_defined_cmap_degs = LinearSegmentedColormap.from_list('mycmap', ["#0000FF", "#3333FF", "#6666FF", "#9999FF", "#B2B2FF", "#CCCCFF", "#E6E6FF", "#E6FFE6", "#CCFFCC", "#B2FFB2", "#99FF99", "#66FF66", "#33FF33", "#00FF00"])

In [4]:
%matplotlib inline 

## Load annotated Figure 1

In [5]:
path_to_h5ad = '../output/anndata_io/Fig1pt1_annotated.h5ad'

In [6]:
adata_d0 = sc.read_h5ad(path_to_h5ad)
adata_d0.uns['log1p']["base"] = None

In [7]:
adata_d0.shape

(22932, 27657)

## Differential expression analysis

### All subset markers

In [ ]:
sc.tl.rank_genes_groups(adata_d0, 'cell_type_subset', method='wilcoxon', use_raw=False)

In [ ]:
writer = pd.ExcelWriter('../output/wilcox_items/adata_d0_wilcox.xlsx', engine='xlsxwriter')

# top 5 differentially expressed genes for each cell type. Change number in brackets to get a more extended gene list
result = adata_d0.uns['rank_genes_groups']
groups = result['names'].dtype.names
pd.DataFrame(
    {group + '_' + key[:1]: result[key][group]
    for group in groups for key in ['names', 'scores', 'logfoldchanges', 'pvals_adj']}).to_excel(writer)

writer.save()

### 18mo vs 02mo

In [ ]:
adata_d0.obs['stage_and_subset'] = ''
adata_d0.obs['stage_and_subset'] = adata_d0.obs[['stage', 'cell_type_subset']].agg('_'.join, axis=1)

In [ ]:
writer = pd.ExcelWriter('adata_18vs02mo_d0_wilcox.xlsx', engine='xlsxwriter')

for subset in ["0:arEC", "1:capEC", "2:venEC", "3:capsFB", "4:intFB", "5:medFB", "6:MEC", "7:vSMC/PC", "8:nmSC","9:Fat", "10:aaTEC1",  "11:aaTEC2",  "12:cTEC", "12:early Pr",         
              "13:mTEC1", "14:mTEC-prol", "15:mTEC2", "16:mimetic(basal)", "17:mimetic(tuft)", "18:mimetic(neuroendo)", "19:mimetic(goblet)", "20:mimetic(microfold)"]:   
    sc.tl.rank_genes_groups(adata_d0, 'stage_and_subset', groups=['18mo_'+subset], reference='02mo_'+subset, method='wilcoxon', use_raw=False)
    result = adata_d0.uns['rank_genes_groups']
    groups = result['names'].dtype.names
    pd.DataFrame(
        {group + '_' + key[:1]: result[key][group]
        for group in groups for key in ['names', 'scores', 'logfoldchanges', 'pvals_adj']}).to_excel(writer, sheet_name=re.search('.*:(.+)', subset).group(1).replace('/', '-'))
        
writer.save()

## Convert wilcoxon results to GSEA ranks

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%R if (!require("pacman")) install.packages("pacman")
%R pacman::p_load(MAST, scales, data.table, openxlsx, ggplot2, ggpubr, RColorBrewer, dichromat, readxl, ggpubr, pheatmap, dplyr, arrow, feather, DelayedArray, HDF5Array, stringr, parallel)

In [ ]:
%%R 

# read in all available excel sheet names

wilcox_results = excel_sheets('adata_18vs02mo_d0_wilcox.xlsx')

rnk_items_list = NULL

for (item in wilcox_results) {
      wilcox_result <- read_excel('adata_18vs02mo_d0_wilcox.xlsx', sheet = item)
      rnk_item = na.omit(wilcox_result[,c(2,3)])
      rnk_item_sorted = rnk_item[order(rnk_item[,2], decreasing = TRUE),]
      colnames(rnk_item_sorted)[1] = '#primerid' # comment out header
      colnames(rnk_item_sorted)[2] = '#rank_score' # comment out header
      rnk_items_list[[item]] = rnk_item_sorted
      write.table(rnk_item_sorted, file = paste0('../output/metadata/gsea_items/input_ranks/wilcox_result_', item, '.rnk'), sep='\t', row.names = FALSE, quote = FALSE)
}

## Dotchart using score and fdr from wilcox

In [ ]:
%%R 

wilcox_results = excel_sheets('../output/wilcox_items/adata_18vs02mo_d0_wilcox.xlsx')
wilcox_results_combined = NULL
for (item in wilcox_results) {
    wilcox_result <- read_excel('adata_18vs02mo_d0_wilcox.xlsx', sheet = item)
    colnames(wilcox_result) <- c('index', 'name', 'score', 'log2_fc', 'p_adj')
    wilcox_result$p_adj[wilcox_result$p_adj == 0] <- min(wilcox_result$p_adj[wilcox_result$p_adj>0])
    wilcox_result$`-log10(p_adj)` = -log(wilcox_result$p_adj, 10)
    wilcox_result$subset = item
    wilcox_result_sorted = wilcox_result[order(wilcox_result$score, decreasing = TRUE),]
    wilcox_result_sorted = wilcox_result_sorted[wilcox_result_sorted$p_adj<=0.05,]
    wilcox_results_combined = bind_rows(wilcox_results_combined, wilcox_result_sorted) # select # of top genes per subset 
}

In [ ]:
%%R

L <- c('Foxn1', 'Dll4', 'Cxcl12', 'Ccl19', 'Ccl21a', 'Ccl25', 'Fgf1', 'Fgf2', 'Fgf7', 'Fgf10', 'Fgf18', 'Fgf21', 'Bmp4', 'Bmp7', 'Flt3l', 'Kitl')
'Foxn1', 'Dll4', 'Cxcl12',	'Ccl19',	'Ccl21a',	'Ccl25','Fgf7', 	'Fgf1',	'Fgf2',		'Fgf10',	'Fgf18',	'Fgf21',	'Bmp4',	'Bmp7',	'Flt3l',	'Kitl'

In [ ]:
%%R 
tt = wilcox_results_combined[wilcox_results_combined$name %in% L,]
tt$name <- factor(tt$name, levels = rev(c('Bmp7',	'Bmp4',	'Flt3l',	'Kitl',  'Fgf18', 'Fgf7', 'Fgf2',	'Fgf10', 'Fgf1','Fgf21', 'Dll4', 'Cxcl12',	'Ccl19',	'Ccl21a',	'Ccl25', 'Foxn1')))
tt$subset <- factor(tt$subset, levels = c('capsFB', 'intFB', 'medFB', 'MEC', 'capEC', 'cTEC', 'mTEC1', 'mTEC-prol', 'mTEC2'))
tt$squishedZ = squish(tt$score, range=c(-3, 3), only.finite=TRUE)

In [ ]:
%%R -w 10.5 -h 10 -u cm

pdf("dotplot_Zscore_d0_L_alt.pdf", width=3.75, height=3.35)
# plot marker genes in a dotplot format 
# use BluetoGreen.14 colorscheme for degs
print(ggdotchart(tt, x='name', y='subset', group = 'subset',rotate=TRUE, color='squishedZ', size = '-log10(p_adj)', sorting='none', xlab = "",  ylab = "") +
scale_color_gradientn(colours = dichromat::colorschemes$BluetoGreen.14) +  
theme_pubr() + theme(legend.position='right', axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1)))

dev.off() #

## Cytoscape summarized results

In [ ]:
%%R

fromNetwork <- read_excel("../output/cytoscape_items/18vs02mo_m5_annotated_NEW.xlsx", sheet="without NAs")
fromNetwork_clean <- select(fromNetwork, -contains(c("::Dataset_Chart", "::Genes", "::GS_DESCR", "::Name", "shared name", "selected", 
                                                     "::GS_Type", "Colouring", "::ES", "::fwer_qvalue", "::pvalue",
                                                      "aaTEC1", "aaTEC2", "Fat", "nmSC"
                                                )))
#      "aaTEC1", "aaTEC2", "Fat", "MEC", "nmSC", "PCvSMC"
fromNetwork_clean$minFDR <- apply(select(fromNetwork_clean, contains(c("EnrichmentMap::fdr_qvalue"))), 1, FUN = min)

fromNetwork_clean = as_tibble(fromNetwork_clean %>% group_by(annotation_BP) %>% 
   slice_min(order_by = minFDR, n=5, with_ties=FALSE))

forDotplot <- as_tibble(lapply(select(fromNetwork_clean, contains(c("gs_size", "name", "annotation_BP", "minFDR")))  , rep, ncol(select(fromNetwork_clean,starts_with("EnrichmentMap::fdr_qvalue")))))

tmpFDR = NULL
FDR=NULL
tmpNES = NULL
NES=NULL
tmp_subset = NULL
subset=NULL


# append (in order) all FDR columns and all NES columns
for (i in c(1:ncol(select(fromNetwork_clean ,starts_with("EnrichmentMap::fdr_qvalue"))))) {
  tmpFDR = select(fromNetwork_clean,starts_with("EnrichmentMap::fdr_qvalue"))[i]
  colnames(tmpFDR) = "FDR"
  FDR = rbind(FDR, tmpFDR)
}

for (i in c(1:ncol(select(fromNetwork_clean,starts_with("EnrichmentMap::NES"))))) {
  tmpNES = select(fromNetwork_clean,starts_with("EnrichmentMap::NES"))[i]
  colnames(tmpNES) = "NES"
  NES = rbind(NES, tmpNES) 
}

forDotplot$NES = NES$NES
forDotplot$FDR = FDR$FDR

forDotplot$subset = rep(str_replace(colnames(select(fromNetwork_clean,starts_with("EnrichmentMap::NES"))), "EnrichmentMap::NES ", ""), each=nrow(fromNetwork_clean))
forDotplot$subset <- str_replace(forDotplot$subset, " - 18mo vs 02mo", "")
#forDotplot$subset <- str_replace(forDotplot$subset, "(", "")

colnames(forDotplot) <- c("gs_size", "name", "group", "minFDR", "NES", "FDR", "subset")
forDotplot$FDR[forDotplot$FDR>0.05] <- NA
forDotplot$FDRtr = -log(forDotplot$FDR+0.00001,10)
print(levels(factor(forDotplot$subset)))
forDotplot$subset <- factor(forDotplot$subset, levels = c("(capsFB)", "(intFB)", "(medFB)", "(MEC)", "(vSMCPC)",  "(arEC)", "(capEC)", "(venEC)",  "(early Pr)", "(cTEC)",  "(mTEC1)", "(mTECprol)", "(mTEC2)", "(mimetic(basal))", "(mimetic(tuft))", "(mimetic(neuroendo))", "(mimetic(goblet))", "(mimetic(microfold))"))

forDotplot = arrange(forDotplot, desc(group), desc(FDR))
print(forDotplot)

In [ ]:
%%R -w 23.5 -h 27 -u cm 

pdf("~/Desktop/dotplot_pathways_d0_top5_NEW_m5.pdf", width=9.5, height=9.5)

print(ggdotchart(forDotplot %>% na.omit(),  x='name', y='subset', group = 'group',  size='FDRtr', rotate=TRUE, color = 'NES', xlab = "", ylab = "", sorting = "none") +
  scale_colour_gradientn(colours = dichromat::colorschemes$BluetoGreen.14) +
  theme_pubclean() + theme(axis.text.x = element_text(face ="bold", angle = 90), axis.text.y = element_text(size =6)) +
  theme(legend.position = "right", panel.background = element_rect(colour = "black",size = 1, linetype = "solid"))) #+ 
dev.off() 